In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Config

#path to file generated by other script will take more than one data set
DATA_FILES = [r"./data/synthetic_data_independent_failures_5.csv"] 

SEQUENCE_LENGTH = 24 * 7 * 2 
STEP_SIZE = 24 * 2
FORECAST_HORIZON = 5
BATCH_SIZE = 32
NUM_EPOCHS = 10

# Set device for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


In [ ]:
# Data processing 
def load_and_combine_data(file_paths):
    """
    Loads multiple CSV files, finds the union of all sensor features, 
    reindexes dataframes to match the full feature set (filling missing sensors with 0.0), 
    and concatenates them into a single DataFrame.
    """
    all_data = []
    all_sensor_features = set()
    
    # Load data and collect all unique sensor feature names
    non_sensor_cols = ['machine_id', "timestamp",'failure_mode', 'is_precursor_period', 'is_final_failure']
    
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path)
            
            # Identify potential sensor features in the current file
            current_sensor_features = [col for col in df.columns if col not in non_sensor_cols]
            print(current_sensor_features)
            all_sensor_features.update(current_sensor_features)
            
            all_data.append(df)
            print(f"Loaded {file_path} with {len(current_sensor_features)} sensor features.")
        except FileNotFoundError:
            print(f"Warning: File '{file_path}' not found. Skipping.")
    
    if not all_data:
        raise FileNotFoundError("No valid data files were loaded.")

    
    sensor_feature_list = sorted(list(all_sensor_features))
    
    
    final_combined_df = pd.DataFrame()
    
    
    full_column_list = non_sensor_cols + sensor_feature_list
    
    for df in all_data:
        
        df_reindexed = df.reindex(columns=full_column_list, fill_value=0.0)
        
        final_combined_df = pd.concat([final_combined_df, df_reindexed], ignore_index=True)

    print(f"\nSuccessfully combined {len(all_data)} files.")
    print(f"Total rows in combined data: {len(final_combined_df)}")
    print(f"Total unique sensor features used: {len(sensor_feature_list)}")
    
    
    return final_combined_df, sensor_feature_list

In [14]:
def create_sequences(data, seq_length, forecast_horizon, step_size):
    sequences = []
    target = []
    for i in range(len(data) - seq_length - forecast_horizon + 1, step_size):
        sequences.append(data[i:i+seq_length])
        target.append(data[i+seq_length: i+seq_length+forecast_horizon])
    return np.array(sequences), np.array(target)

In [ ]:
def prepare_data(df, sensor_features):
    """Loads, cleans, labels, and scales the data."""
    sensor_data = df[sensor_features].values
   
    # 2. Standardization
    scaler = StandardScaler()
    sensor_data_scaled = scaler.fit_transform(sensor_data)
    
    # 3. Create Sequences
    X_seq, Y_seq = create_sequences(sensor_data_scaled, SEQUENCE_LENGTH, FORECAST_HORIZON, STEP_SIZE)
    
    print(f"\n--- Data Preparation Complete ---")
    print(f"Total time points in raw data: {len(df)}")
    print(f"Total sequences created: {len(X_seq)}")
    print(f"Sequence shape (time steps, features): {X_seq.shape[1:]}")
    print(f"Label distribution: {pd.Series(Y_seq).value_counts().sort_index()}")
    
    return X_seq, Y_seq

In [5]:
# PYTORCH Model for fun or more like testing
class TimeDataset(Dataset):
    """Custom PyTorch Dataset for time-series sequences."""
    def __init__(self, X_data, y_data):

        self.X_data = torch.tensor(X_data, dtype=torch.float32).permute(0, 2, 1)
        self.y_data = torch.tensor(y_data, dtype=torch.long)
        
    def __len__(self):
        return len(self.y_data)
    
    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [6]:
class CNNClassifier(nn.Module):
    """
    5-layer 1D CNN for time-series classification built with PyTorch.
    Input channels adapts based on the number of sensor features.
    """
    def __init__(self, input_channels, num_classes):
        super(CNNClassifier, self).__init__()
        
        # Layer 1: Conv1D (Large filter for high-level feature extraction)
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=12, padding='same')
        
        # Layer 2: Conv1D (Smaller filter for finer patterns)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=6, padding='same')
        
        # Layer 3: Max Pooling (Downsampling)
        self.pool = nn.MaxPool1d(kernel_size=4)
        
        # Layer 4: Conv1D (Deep feature extraction)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding='same')
        
        # Layer 5: Global Average Pooling (Summarizes the time-series output)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.3)
        
        # Final fully connected layer
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        
        # Apply Global Average Pooling
        x = self.global_pool(x)
        
        # Flatten the tensor from (batch, features, 1) to (batch, features)
        x = x.view(x.size(0), -1) 
        
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [7]:
# --- 3. TRAINING AND EVALUATION ---

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    """PyTorch training loop."""
    model.train()
    print(f"\n--- Starting PyTorch Model Training ({num_epochs} Epochs) ---")
    
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, test_loader):
    """PyTorch evaluation loop and metric calculation."""
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(DEVICE)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            
    return y_true, y_pred

In [13]:
try:
    data, sensor_features = load_and_combine_data(DATA_FILES)
except FileNotFoundError as e:
    print(f"Fatal Error: {e}")

['timestamp', 'T_internal_sensor', 'V_sensor']
Loaded ./data/synthetic_data_independent_failures_5.csv with 3 sensor features.

Successfully combined 1 files.
Total rows in combined data: 131400
Total unique sensor features used: 3


In [ ]:
X_seq, Y_seq = prepare_data(data, sensor_features)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X_seq, Y_seq, test_size=0.2, random_state=42, stratify=Y_seq)

In [ ]:
# Imbalanced Data
train_labels_series = pd.Series(Y_train)
class_counts = train_labels_series.value_counts().sort_index()
total_samples = len(Y_train)


class_weights = total_samples / (NUM_CLASSES * class_counts)


class_weights_tensor = torch.tensor(class_weights.values, dtype=torch.float32).to(DEVICE)

print("\nCalculated Class Weights (Higher = More Important):")
for i, weight in enumerate(class_weights_tensor):
    print(f"Class {i} Weight: {weight:.2f}")

In [ ]:
# Create PyTorch Datasets and DataLoaders
train_dataset = TimeDataset(X_train, Y_train)
test_dataset = TimeDataset(X_test, Y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Build the PyTorch CNN model
input_channels = X_train.shape[2] 
model = CNNClassifier(input_channels, NUM_CLASSES).to(DEVICE)

print("\nModel Architecture:")
print(model)

In [ ]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, criterion, optimizer, NUM_EPOCHS)

In [ ]:
# Evaluate the model
print("\n--- Evaluating Model Performance ---")
y_true, y_pred = evaluate_model(model, test_loader)

# # Basic classification report
# print("\nClassification Report (PyTorch):")
# report = classification_report(y_true, y_pred, 
#                                 target_names=['Normal (0)', 'Precursor (1)', 'Failure (2)'],
#                                 labels=[0, 1, 2])
# print(report)

# print("\nConfusion Matrix:")
# print(confusion_matrix(y_true, y_pred))